# Data Preprocessing

In [1]:
!pip install polars

import pandas as pd
import polars as pl
from gensim.test.utils import common_texts
from gensim.models import Word2Vec


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 13.7 MB/s eta 0:00:00


In [2]:
train = pd.read_parquet('../input/otto-full-optimized-memory-footprint/train.parquet')
test = pd.read_parquet('../input/otto-full-optimized-memory-footprint/test.parquet')

In [3]:
def label_encoding(df):

    #set start = 2 to reserve 0 for mask and 1 for pad
    df['aid+2'] = df['aid']+2
    df['aid'] = df['aid+2']

    df = df.drop(['aid+2'], axis = 1)
    return df

train = label_encoding(train)
test = label_encoding(test)

In [4]:
train.loc[len(train.index)]  = [19000000, 0,0,0]

In [5]:
train.session = train.session.astype('int32')
train.ts = train.ts.astype('int32')
train.type =train.type.astype('uint8')

In [6]:
train = pl.DataFrame(train)
test = pl.DataFrame(test)



sentences_df = pl.concat([train, test]).groupby('session').agg(
    pl.col('aid').alias('sentence')
)

sentences = sentences_df['sentence'].to_list()

# Training a word2vec model

In [7]:
%%time

w2vec = Word2Vec(sentences=sentences, vector_size=32, min_count=1, workers=4)
w2vec.save("/kaggle/working/word2vec.model")

CPU times: user 1h 32min 38s, sys: 23.2 s, total: 1h 33min 2s
Wall time: 29min 55s
